In [1]:
import numpy as np

class HungarianAlgorithm:
    def __init__(self, cost_matrix):
        self.cost_matrix = np.array(cost_matrix)
        self.n, self.m = self.cost_matrix.shape
        self.max_dim = max(self.n, self.m)
        # Приведение к квадратной матрице, если это не так
        self.cost_matrix = np.pad(self.cost_matrix, ((0, self.max_dim - self.n), (0, self.max_dim - self.m)), mode='constant', constant_values=0)
        self.n = self.max_dim

    def _reduce_matrix(self):
        """Вычитание минимального элемента из каждой строки и каждого столбца."""
        for i in range(self.n):
            self.cost_matrix[i] -= np.min(self.cost_matrix[i])
        for j in range(self.n):
            self.cost_matrix[:, j] -= np.min(self.cost_matrix[:, j])

    def _find_zeros(self):
        """Находит все нули в текущей матрице."""
        return np.where(self.cost_matrix == 0)

    def _cover_zeros(self):
        """Покрытие минимальным количеством строк и столбцов."""
        covered_rows = np.zeros(self.n, dtype=bool)
        covered_cols = np.zeros(self.n, dtype=bool)
        zero_locations = self._find_zeros()

        row_mark = np.zeros(self.n, dtype=bool)
        col_mark = np.zeros(self.n, dtype=bool)

        for r, c in zip(*zero_locations):
            if not row_mark[r] and not col_mark[c]:
                row_mark[r] = True
                col_mark[c] = True

        for r in range(self.n):
            if not row_mark[r]:
                covered_rows[r] = True

        # Покрываем столбцы, содержащие отмеченные строки
        while True:
            old_cols = covered_cols.copy()
            for r in range(self.n):
                if covered_rows[r]:
                    for c in range(self.n):
                        if self.cost_matrix[r, c] == 0:
                            covered_cols[c] = True
            for c in range(self.n):
                if covered_cols[c]:
                    for r in range(self.n):
                        if self.cost_matrix[r, c] == 0:
                            covered_rows[r] = True
            if np.array_equal(old_cols, covered_cols):
                break

        return covered_rows, covered_cols

    def _adjust_matrix(self, row_covered, col_covered):
        """Корректировка значений матрицы."""
        uncovered_values = self.cost_matrix[~row_covered][:, ~col_covered]
        min_val = np.min(uncovered_values)
        # Вычитаем минимальное значение из всех непокрытых элементов
        self.cost_matrix[~row_covered, :] -= min_val
        # Добавляем минимальное значение к элементам в пересечениях покрытых строк и столбцов
        self.cost_matrix[:, col_covered] += min_val

    def solve(self):
        self._reduce_matrix()

        while True:
            row_covered, col_covered = self._cover_zeros()
            covered_count = np.sum(row_covered) + np.sum(col_covered)
            if covered_count >= self.n:
                break
            self._adjust_matrix(row_covered, col_covered)

        result = []
        zero_locations = self._find_zeros()
        used_rows = set()
        used_cols = set()
        for r, c in zip(*zero_locations):
            if r not in used_rows and c not in used_cols:
                result.append((r, c))
                used_rows.add(r)
                used_cols.add(c)

        return [(r, c, self.cost_matrix[r, c]) for r, c in result]


In [ ]:
cost_matrix = [
    [4, 2, 8],
    [2, 4, 6],
    [8, 6, 4]
]

hungarian = HungarianAlgorithm(cost_matrix)
assignments = hungarian.solve()

print("Оптимальные назначения (строка, столбец, стоимость):")
for row, col, cost in assignments:
    print(f"Задача {row} назначена работнику {col} с стоимостью {cost}")

In [ ]:
Описание методов: _reduce_matrix: Вычитает минимальный элемент из каждой строки и каждого столбца. Гарантирует, что в каждой строке и столбце есть хотя бы один 0. _cover_zeros: Находит минимальное количество строк и столбцов, покрывающих все нули. _adjust_matrix: Модифицирует матрицу, если количество покрытий меньше 𝑛 n. solve: Основной метод. Находит оптимальное назначение с минимальной стоимостью.

In [ ]:
Сложность алгоритма: Основные этапы: Уменьшение матрицы — 𝑂 ( 𝑛 2 ) O(n 2 ), так как требуется обойти каждый элемент. Покрытие строками и столбцами — 𝑂 ( 𝑛 2 ) O(n 2 ) для поиска пересечений. Корректировка матрицы — 𝑂 ( 𝑛 2 ) O(n 2 ). Итерации продолжаются максимум 𝑛 n раз. Общая временная сложность: 𝑂 ( 𝑛 3 ) O(n 3 ).

In [ ]:
Пространственная сложность: Матрица затрат: Хранится квадратная матрица размером 𝑛 × 𝑛 n×n, что занимает 𝑂 ( 𝑛 2 ) O(n 2 ) памяти. Вспомогательные массивы: Покрытие строк и столбцов требует два булевых массива длины 𝑛 n, то есть 𝑂 ( 𝑛 ) O(n). Общая пространственная сложность: 𝑂 ( 𝑛 2 ) O(n 2 ).